In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv("./Dataset/Recommendation Loans.csv")

- ***Click Matrix***

    - > No. of Click should be between 0-5
    - > If not Recommended then it will be NAN 

-> What Could be the max times the Loan could be clicked!

- **Form Matrix**

    - >No of forms filled Any of the 1 form filled from M form Recommended
    - >If not Recommended then it will be NaN.
-> Is Filling a single form is allowd Also if user has not clicked the loan does he able to fill the form

In [3]:
# Create an empty dataframe to store the transformed data
new_df = pd.DataFrame(columns=['User Id', 'Loan', 'Recommendation_Rank', 'CTR', 'FFR'])

# Iterate over each row in the initial dataframe
for index, row in df.iterrows():
    # Extract User Id and Recommendations list
    user_id = row['UserId']
    recommendations = row['Recommendations'][1:-1].split(',')
    form_filling = random.randint(0, 5)
    # Iterate over each loan in the recommendations list
    for indx,loan in enumerate(recommendations, start=1):
        # Append a new row to the new dataframe for each loan
        click_filling  = random.randint(0, 5)
        if indx == form_filling and click_filling > 0:
            new_df = pd.concat([new_df, pd.DataFrame({'User Id': [user_id], 'Loan': [loan], 'Recommendation_Rank':[indx], 'CTR':[click_filling],'FFR':[1]})], ignore_index=True)
        else:    
            new_df = pd.concat([new_df, pd.DataFrame({'User Id': [user_id], 'Loan': [loan], 'Recommendation_Rank':[indx], 'CTR':[click_filling],'FFR':[np.nan]})], ignore_index=True)

# Convert User Id column to integer type
new_df['User Id'] = new_df['User Id'].astype(int)


In [4]:
new_df.head(40)

,User Id,Loan,Recommendation_Rank,CTR,FFR
0,1,19,1,5,NaN
1,1,20,2,2,NaN
2,1,10,3,5,NaN
3,1,13,4,4,NaN
4,1,6,5,0,NaN
5,2,3,1,2,NaN
6,2,14,2,2,NaN
7,2,2,3,2,NaN
8,2,7,4,2,NaN
9,2,15,5,3,NaN


In [6]:
new_df.shape

(292160, 5)

In [5]:
new_df.head()

,User Id,Loan,Recommendation_Rank,CTR,FFR
0,1,19,1,5,NaN
1,1,20,2,2,NaN
2,1,10,3,5,NaN
3,1,13,4,4,1.0
4,1,6,5,0,NaN


In [8]:
new_df.to_csv("LinearData_CTR_Form_fill_rate_Data.csv",index=False)

In [3]:
new_df = pd.read_csv("./LinearData_CTR_Form_fill_rate_Data_V-2.csv")

In [4]:
new_df.head()

,User Id,Loan,Recommendation_Rank,CTR,FFR
0,1,19,1,5,NaN
1,1,20,2,2,NaN
2,1,10,3,5,NaN
3,1,13,4,4,1.0
4,1,6,5,0,NaN


# Calculating the Relevancy of the Loans

In [11]:
new_df['Relevancy'] = new_df['CTR']

In [18]:
new_df.head()

,User Id,Loan,Recommendation_Rank,CTR,FFR,Relevancy
0,1,19,1,5,NaN,True
1,1,20,2,2,NaN,True
2,1,10,3,5,NaN,True
3,1,13,4,4,1.0,True
4,1,6,5,0,NaN,False


In [17]:
new_df['Relevancy'] = new_df['Relevancy'].apply(lambda x: x>0 )

In [20]:
new_df['Relevancy'] = new_df['Relevancy'].astype(int)

In [21]:
new_df.head()

,User Id,Loan,Recommendation_Rank,CTR,FFR,Relevancy
0,1,19,1,5,NaN,1
1,1,20,2,2,NaN,1
2,1,10,3,5,NaN,1
3,1,13,4,4,1.0,1
4,1,6,5,0,NaN,0


In [73]:
def MAP_AT_K(new_df,k):
    flag=0
    map_K = 0
    for _, group in new_df.groupby("User Id"):
        group = pd.DataFrame(group, columns=new_df.columns)
        precision = []
        calculati = []
        count =0
        relevency = 0
        breaked = 0
        for index, row in group.iterrows():
            relevancy_index = group['Relevancy'][index]
            if relevancy_index!=0:
                count += 1
            Preci = count/group['Recommendation_Rank'][index]
            precision.append(Preci)
            calculati.append(Preci*group['Relevancy'][index])
            relevency = sum(group['Relevancy'])
            if index >= k:
                breaked=1
                break
        precigency = sum(calculati)
        AvgPrecision = 0
        if relevency>0:
            AvgPrecision = precigency/relevency
        # group['Precision']= precision
        # group['calculati']= calculati
        # group['AvgPrecision']= AvgPrecision
        map_K += AvgPrecision
        breaked = 0
    return map_K/new_df['User Id'].nunique()
    
f"MAP@5 for Based on {flag} users",MAP_AT_K(new_df,3) # map_K/flag

('MAP@5 for Based on 11 users', 0.20042100219064202)

In [ ]:
('MAP@5 for Based on 11 users', 0.8876697842505309)

In [70]:
 

def MAP_AT_K_Gen(new_df, k):
    # Initialize variables
    map_K = 0
    total_users = new_df['User Id'].nunique()
    
    # Group by User Id
    grouped = new_df.groupby("User Id")
    flag = 0
    for _, group in grouped:
        # Sort group by Recommendation_Rank
        group = group.sort_values(by='Recommendation_Rank')
        
        # Filter relevant items with Recommendation_Rank <= k
        group_top_k = group[group['Recommendation_Rank'] <= k]
        
        # Calculate cumulative relevancy at k
        group_top_k['Cumulative_Relevancy'] = group_top_k['Relevancy'].cumsum()
        
        # Calculate precision at k
        group_top_k['Precision_at_K'] = group_top_k['Cumulative_Relevancy'] / group_top_k['Recommendation_Rank']
        
        # Calculate average precision at k
        avg_precision_at_k = group_top_k['Precision_at_K'].sum() / min(k, len(group))
        
        # Update MAP at k
        map_K += avg_precision_at_k
        if flag=10:
            break
        flag += 1
    # Compute MAP at k
    return map_K / total_users

f"MAP@5 for Based on {flag} users",MAP_AT_K_Gen(new_df,5) # map_K/flag

('MAP@5 for Based on 11 users', 0.8345108844469085)

In [69]:
new_df.head()

,User Id,Loan,Recommendation_Rank,CTR,FFR,Relevancy
0,1,19,1,5,NaN,1
1,1,20,2,2,NaN,1
2,1,10,3,5,NaN,1
3,1,13,4,4,1.0,1
4,1,6,5,0,NaN,0


In [9]:
new_df.drop("Relevancy",axis=1,inplace=True)

In [ ]:
>>> df.apply(lambda x: [1, 2], axis=1, result_type='expand')
 